In [121]:
import numpy as np
import math

import gurobipy as gp
from gurobipy import GRB, tuplelist

import pandas as pd

In [122]:
idx_x = np.arange(1,9,1)
set_x = tuplelist([i for i in idx_x])

In [123]:
def variables_ori(m):
    # X
    x = m.addVars(set_x, name='x') 
    x[1].LB, x[1].UB = 100, 10000

    for i in [2,3]:
        x[i].LB, x[i].UB = 1000, 10000

    for i in [4,5,6,7,8]:
        x[i].LB, x[i].UB = 10, 1000

    return x

In [124]:
def objective_ori(m,x):
    sum_x = sum(x[i] for i in [1,2,3])

    m.setObjective(sum_x, GRB.MINIMIZE)

In [125]:
def constraints_ori(m,x):
    m.addConstr(0.0025*(x[4] + x[6]) - 1 <= 0)
    m.addConstr(0.0025*(-x[4] + x[5] + x[7]) - 1 <= 0)
    m.addConstr(0.01*(-x[5] + x[8]) - 1 <= 0)
    m.addConstr(100*x[1] - x[1]*x[6] + 833.33252*x[4] - 83333.333 <= 0)
    m.addConstr(x[2]*x[4] - x[2]*x[7] - 1250*x[4] + 1250*x[5] <= 0)
    m.addConstr(x[3]*x[5] - x[3]*x[8] - 2500*x[5] + 1250000 <= 0)

In [126]:
def constraints_mdt(m,x):
    pairs = tuplelist([(1,6),(2,4),(2,7),(3,5),(3,8)])
    # pairs = tuplelist([(i[1], i[0]) for i in pairs]) # Flip the discretization
    
    w = m.addVars(pairs, name='w')

    m.addConstr(0.0025*(x[4] + x[6]) - 1 <= 0)
    m.addConstr(0.0025*(-x[4] + x[5] + x[7]) - 1 <= 0)
    m.addConstr(0.01*(-x[5] + x[8]) - 1 <= 0)
    m.addConstr(100*x[1] - w[1,6] + 833.33252*x[4] - 83333.333 <= 0)
    m.addConstr(w[2,4] - w[2,7] - 1250*x[4] + 1250*x[5] <= 0)
    m.addConstr(w[3,5] - w[3,8] - 2500*x[5] + 1250000 <= 0)

    return w,pairs

In [127]:
def constraints_mdt_flip(m,x):
    pairs = tuplelist([(1,6),(2,4),(2,7),(3,5),(3,8)])
    pairs = tuplelist([(i[1], i[0]) for i in pairs]) # Flip the discretization
    
    w = m.addVars(pairs, name='w')

    m.addConstr(0.0025*(x[4] + x[6]) - 1 <= 0)
    m.addConstr(0.0025*(-x[4] + x[5] + x[7]) - 1 <= 0)
    m.addConstr(0.01*(-x[5] + x[8]) - 1 <= 0)
    m.addConstr(100*x[1] - w[6,1] + 833.33252*x[4] - 83333.333 <= 0)
    m.addConstr(w[4,2] - w[7,2] - 1250*x[4] + 1250*x[5] <= 0)
    m.addConstr(w[5,3] - w[8,3] - 2500*x[5] + 1250000 <= 0)
    
    y = m.addVar(vtype=GRB.BINARY, name='y')

    m.addConstr(x[1] <= y*x[1].UB)
    m.addConstr(x[1] >= y*x[1].LB)
    # m.addConstr(x[6] <= (1-y)*x[6].UB)

    return w,pairs

In [128]:
from collections import Counter
# To find the unique elements from the tuple using the counter

def unique_num(numbers):
    # this will take only unique numbers from the tuple
    return tuple(Counter(numbers).keys())

In [129]:
def relax_mdt(m,x,p=0,P=0):
    # w,pairs = constraints_mdt(m,x)
    w,pairs = constraints_mdt_flip(m,x)

    pairs
    left_set = unique_num([i[0] for i in pairs])
    right_set = unique_num([i[1] for i in pairs])

    set_l = range(p,P+1)
    set_k = range(10)
    set_kl = tuplelist([(i,j,k,l) for l in set_l for k in set_k for i,j in pairs])
    set_z = tuplelist([(i,k,l) for l in set_l for k in set_k for i in left_set])
    # w,pairs = constraints_mdt(m,x)
    

    # Here, 'left_set' are the variables that are discretized and 'right_set' are the variables that are continuous
    delta_w = m.addVars(pairs, lb=-GRB.INFINITY, ub=GRB.INFINITY, name='delta_w')
    delta_x1 = m.addVars(left_set, lb=0, ub=10**p, name='delta_x1')
    
    # Indexed continuous variables (hat_x_k) and binary variables (z_k)
    hat_x = m.addVars(set_kl, lb=-GRB.INFINITY, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name='hat_x')
    z = m.addVars(set_z, vtype=GRB.BINARY, name='z')

    m.update()
        
    m.addConstrs((w[i,j] == gp.quicksum(gp.quicksum(10**l * k * hat_x[i,j,k,l] for k in set_k) for l in set_l) + delta_w[i,j] for i,j in pairs))

    m.addConstrs((x[i] == gp.quicksum(gp.quicksum(10**l * k * z[i,k,l] for k in set_k) for l in set_l) + delta_x1[i] for i in left_set))

    m.addConstrs((x[j] == gp.quicksum(hat_x[i,j,k,l] for k in set_k) for l in set_l for i,j in pairs))

    m.addConstrs((hat_x[i,j,k,l] >= x[j].LB * z[i,k,l] for i,j,k,l in set_kl))
    m.addConstrs((hat_x[i,j,k,l] <= x[j].UB * z[i,k,l] for i,j,k,l in set_kl))

    m.addConstrs((z.sum(i,'*',l) == 1 for i,k,l in set_z))

    m.addConstrs((delta_w[i,j] >= x[j].LB * delta_x1[i] for i,j in pairs))
    m.addConstrs((delta_w[i,j] <= x[j].UB * delta_x1[i] for i,j in pairs))

    m.addConstrs((delta_w[i,j] <= (x[j] - x[j].LB) * 10**p + x[j].LB * delta_x1[i] for i,j in pairs))
    m.addConstrs((delta_w[i,j] >= (x[j] - x[j].UB) * 10**p + x[j].UB * delta_x1[i] for i,j in pairs))

    return w, delta_w, delta_x1, hat_x, z, pairs   # output variables for main program

In [130]:
def var_values(y,mult=1):
    z = []
    for v in y.values():
        z.append(round(v.X*mult,2))
    return z

In [131]:
def optimize_mdt(p,P):
    m = gp.Model('MDT_P3')
    x = variables_ori(m)
    objective_ori(m,x)
    m.update()

    w, delta_w, delta_x1, hat_x, z, pairs = relax_mdt(m,x,p,P)
    m.update()

    m.Params.OutputFlag = 1
    m.optimize()

    xx = var_values(x)
    return xx,m.ObjVal, m.Runtime, m.NumVars, m.NumConstrs

In [132]:
P = 3
p = 3
df = pd.DataFrame(columns=['p','Obj_value','x','Computed in'])
for i in range(P,p-1,-1):
    x,objval,runtime,nvar,nconst = optimize_mdt(i,P)
    df2 = pd.DataFrame({'p':[i], 'Obj_value':[objval], 'x':[x], 'Computed in':[runtime], 'Num. Vars':[nvar], 'Num. Constrs':[nconst]},index=[i])
    df = pd.concat([df,df2])

df.set_index(['p'])
print(df)


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (26120.2))

CPU model: AMD Ryzen 7 2700X Eight-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 193 rows, 124 columns and 936 nonzeros
Model fingerprint: 0xf4d24513
Variable types: 73 continuous, 51 integer (51 binary)
Coefficient statistics:
  Matrix range     [3e-03, 1e+04]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+04]
  RHS range        [1e+00, 1e+07]
Presolve removed 168 rows and 111 columns
Presolve time: 0.00s
Presolved: 25 rows, 13 columns, 64 nonzeros
Variable types: 13 continuous, 0 integer (0 binary)

Root relaxation: objective 2.529514e+03, 15 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    2529.5137097 2529.

In [133]:
df.to_csv('result_p3.csv', sep='\t', index=False)

In [134]:
m = gp.Model('MDT')
x = variables_ori(m)
# constraints_mdt(m,x)
objective_ori(m,x)
m.update()
# P = int(math.log10(1000))
w, delta_w, delta_x1, hat_x, z, pairs = relax_mdt(m,x,p=-1,P=3)

m.update()
m.write('P3.lp')

m.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (26120.2))

CPU model: AMD Ryzen 7 2700X Eight-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 813 rows, 524 columns and 4316 nonzeros
Model fingerprint: 0xb6b245fc
Variable types: 273 continuous, 251 integer (251 binary)
Coefficient statistics:
  Matrix range     [3e-03, 1e+04]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e-01, 1e+04]
  RHS range        [1e+00, 1e+06]


Presolve removed 378 rows and 142 columns
Presolve time: 0.00s
Presolved: 435 rows, 382 columns, 1521 nonzeros
Variable types: 201 continuous, 181 integer (181 binary)
Found heuristic solution: objective 11097.498193

Root relaxation: objective 2.696561e+03, 303 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 2696.56068    0   14 11097.4982 2696.56068  75.7%     -    0s
     0     0 2771.13077    0   15 11097.4982 2771.13077  75.0%     -    0s
H    0     0                    10938.519765 2771.13077  74.7%     -    0s
     0     0 2771.13077    0   13 10938.5198 2771.13077  74.7%     -    0s
H    0     0                    10849.958326 2790.30593  74.3%     -    0s
     0     0 2790.30593    0   15 10849.9583 2790.30593  74.3%     -    0s
     0     0 2793.51172    0   22 10849.9583 2793.51172  74.3%     -    0s
H    0     0     

In [135]:
m.Runtime

6.828999996185303

In [136]:
m.Runtime

6.828999996185303

In [137]:
def optimize_problem(p,P):
    m = gp.Model('MDT')
    x = variables_ori(m)
    m.update()
    w = relax_mdt(m,x)
    
    

In [138]:
## Defining the upper bound
m = gp.Model('P3')
x = variables_ori(m)
m.update()
objective_ori(m,x)
constraints_ori(m,x)

m.Params.OutputFlag = 0
m.Params.NonConvex = 2
m.optimize()

x_res = [(i, x[i].X) for i in set_x]
df = pd.DataFrame(x_res, columns=['Index', 'x_upper'])
df = df.set_index('Index')

In [139]:
m.ObjVal, m.Runtime, m.NumVars, m.NumConstrs

(7049.248020538567, 0.06999993324279785, 8, 3)

In [140]:
x_res

[(1, 579.3066844257957),
 (2, 1359.9706680592556),
 (3, 5109.9706680535155),
 (4, 182.01769958088101),
 (5, 295.60117327799134),
 (6, 217.98230041878287),
 (7, 286.4165263027629),
 (8, 395.60117327795905)]

In [141]:
print(df)

           x_upper
Index             
1       579.306684
2      1359.970668
3      5109.970668
4       182.017700
5       295.601173
6       217.982300
7       286.416526
8       395.601173
